In [ ]:
import argparse

import cv2
from pupil_detectors import Detector2D

from pye3d.detector_3d import CameraModel, Detector3D, DetectorMode
import matplotlib.pyplot as plt

In [ ]:
detector_2d = Detector2D()
# create pye3D detector
camera = CameraModel(focal_length=228.57, resolution=[400, 640])
detector_3d = Detector3D(camera=camera, long_term_mode=DetectorMode.blocking)

In [ ]:
video_path = "SUMMARY_gaze_2021-11-01_11-25-26-678000_right_eye.mp4"
cap = cv2.VideoCapture(video_path)

FPS = 30
COUNTER = 0

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Read and display frames in a loop
while True:

    COUNTER += 1
    
    ret, frame = cap.read()  # Read one frame
    if not ret:
        print("End of video or can't fetch frame.")
        break

    # read video frame as numpy array
    grayscale_array = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # run 2D detector on video frame
    result_2d = detector_2d.detect(grayscale_array)
    result_2d["timestamp"] = COUNTER / FPS
    # pass 2D detection result to 3D detector
    result_3d = detector_3d.update_and_detect(result_2d)
    ellipse_3d = result_3d["ellipse"]

    confidence = result_3d["confidence"]

    confidence = f"Confidence: {confidence:.2f}"
    print(result_3d.keys())

    cv2.ellipse(frame,
                tuple(int(v) for v in ellipse_3d["center"]),
                tuple(int(v / 2) for v in ellipse_3d["axes"]),
                ellipse_3d["angle"],
                0,
                360,  # start/end angle for drawing
                (255, 0, 255),  # color (BGR): red
               )
    plt.title(confidence)
    plt.imshow(frame)
    plt.show()

    